# SQL commands - 

## Remove Existing Table
- DROP TABLE IF EXISTS sales;

## Create Table
- CREATE TABLE sales (
    id         INT PRIMARY KEY,
    salesperson VARCHAR(100) NOT NULL,
    region      VARCHAR(50)  NOT NULL,
    sale_date   DATE         NOT NULL,
    amount      DECIMAL(10,2) NOT NULL
);

## Sample Data
- INSERT INTO sales (id, salesperson, region, sale_date, amount) VALUES
(1, 'Alice', 'East', '2025-08-01', 200),
(2, 'Bob',   'East', '2025-08-02', 150),
(3, 'Alice', 'West', '2025-08-03', 300),
(4, 'Bob',   'East', '2025-08-04', 250),
(5, 'Alice', 'East', '2025-08-05', 100),
(6, 'Bob',   'East', '2025-08-06', 250),   -- tie with Bob's 250
(7, 'Carol', 'East', '2025-08-02', 150),   -- tie with Bob's 150
(8, 'Dan',   'West', '2025-08-04', 300);   -- tie with Alice's 300 (West)

## Check
- SELECT * FROM sales ORDER BY id;

## VIEW1
### Region-level summary (non-updatable: uses aggregates)
- CREATE OR REPLACE VIEW region_sales_summary AS
SELECT
    region,
    SUM(amount) AS total_sales,
    COUNT(*)    AS num_sales,
    AVG(amount) AS avg_amount
FROM sales
GROUP BY region;
#### Use the View
- SELECT * FROM region_sales_summary ORDER BY region;

### VIEW2
#### Salesperson-level KPIs (non-updatable: aggregates)
- CREATE OR REPLACE VIEW salesperson_totals AS
SELECT
    salesperson,
    SUM(amount) AS total_sales,
    COUNT(*)    AS num_sales,
    MIN(sale_date) AS first_sale_date,
    MAX(sale_date) AS last_sale_date
FROM sales
GROUP BY salesperson;
#### See the View
- SELECT * FROM salesperson_totals ORDER BY total_sales DESC;

###  read/update through this view (example read):
- SELECT * FROM east_sales ORDER BY sale_date;
- Example update via updatable view (optional; comment out if not desired):
    - UPDATE east_sales SET amount = amount + 10 WHERE id = 2;
    - SELECT * FROM east_sales WHERE id = 2;

### View defined over a subquery (inline view inside the view)
- CREATE OR REPLACE VIEW top_sellers AS
SELECT *
FROM (
    SELECT salesperson, SUM(amount) AS total_sales
    FROM sales
    GROUP BY salesperson
) t
WHERE t.total_sales > 300;
#### See the View
- SELECT * FROM top_sellers ORDER BY total_sales DESC;

## SUBQUERIES
### 2A - Subquery in WHERE/HAVING:
- Salespeople whose total sales exceed the overall average sale amount
- SELECT salesperson
FROM sales
GROUP BY salesperson
HAVING SUM(amount) > (
    SELECT AVG(amount)
    FROM sales
);

### 2B  Subquery in FROM (inline view):
- Build per-person totals, then filter in the outer query
- SELECT *
FROM (
    SELECT salesperson, SUM(amount) AS total_sales
    FROM sales
    GROUP BY salesperson
) AS sales_totals
WHERE total_sales > 300
ORDER BY total_sales DESC;


### 2C Subquery in SELECT (scalar correlated subquery):
- Show each sale alongside the average amount for its region
- SELECT
    s1.id,
    s1.salesperson,
    s1.region,
    s1.amount,
    (
        SELECT AVG(s2.amount)
        FROM sales AS s2
        WHERE s2.region = s1.region
    ) AS avg_in_region
FROM sales AS s1
ORDER BY s1.region, s1.amount DESC;

### 2D) EXISTS (semi-join style):
- Salespeople who have at least one sale > the average for their region
SELECT DISTINCT s1.salesperson
FROM sales AS s1
WHERE EXISTS (
    SELECT 1
    FROM sales AS s2
    WHERE s2.region = s1.region
    GROUP BY s2.region
    HAVING MAX(s1.amount) > AVG(s2.amount)
)
ORDER BY s1.salesperson;

### 2E) Correlated subquery in WHERE:
- Latest sale (by date) for each salesperson
- SELECT *
FROM sales AS s1
WHERE s1.sale_date = (
    SELECT MAX(s2.sale_date)
    FROM sales AS s2
    WHERE s2.salesperson = s1.salesperson
)
ORDER BY salesperson, sale_date;